In [8]:
#| export
print_flag                    = None
check_memory_usage            = None
time_flag                     = None
show_plots                    = None
reset_kernel                  = None
check_parameters              = True
cuda_device                   = None
enc_artifact                  = "MVP:v93"

In [9]:
#| export
print_flag                    = True  if print_flag is None else print_flag
check_memory_usage            = True  if check_memory_usage is None else check_memory_usage
time_flag                     = True  if time_flag is None else time_flag
show_plots                    = False if show_plots is None else show_plots
reset_kernel                  = False  if reset_kernel is None else reset_kernel
cuda_device                   = 0 if  cuda_device is None else cuda_device
enc_artifact                  = "MVP:latest" if enc_artifact is None else enc_artifact

In [10]:
if check_parameters:
    print("--- Check parameters ---")
    print(
        "print_flag:", print_flag,
        "check_memory_usage", check_memory_usage,
        "time_flag:", time_flag,
        "window_size_percentage:" , window_size_percentage,
        "show_plots:",show_plots,
        "reset_kernel:",reset_kernel,
        "pre_configured_case:",pre_configured_case,
        "case_id:",case_id,
        "frequency_factor:", frequency_factor, 
        "frequency_factor_change_alias", frequency_factor_change_alias,
        "cuda_device", cuda_device
)

--- Check parameters ---
print_flag: True check_memory_usage True time_flag: True window_size_percentage: False show_plots: False reset_kernel: False pre_configured_case: True case_id: 7 frequency_factor: 1 frequency_factor_change_alias True cuda_device 0


In [11]:
#| export
# This is only needed if the notebook is run in VSCode
import sys
import dvats.utils as ut
if '--vscode' in sys.argv:
    print("Executing inside vscode")
    ut.DisplayHandle.update = ut.update_patch

In [12]:
#| export
import dvats.config as cfg_

# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [13]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [14]:
config, job_type = get_artifact_config_embeddings(print_flag = False)

In [ ]:
aux = config.enc_artifact

In [16]:
cfg_.show_attrdict(config)

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/mvp:latest
input_ar: None
cpu: False


### Show configuration artifact

In [ ]:
for key, value in config.items():
    print(f"{key}: {value}")

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest
input_ar: None
cpu: False


## Build W&B artifact

In [ ]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03a_embeddings"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03a_embeddings


In [ ]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


'stream.Stream' object attribute 'write' is read-only


## Get trained model artifact

### Build artifact selector
> Botch to use artifacts offline

In [ ]:
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [ ]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [ ]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb:   1 of 1 files downloaded.  


## Get dataset artifact from W&B
### Restore the dataset artifact used for training the encoder. 
> Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that it matches with the dataset that we want to reduce.

In [ ]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'solar_4_seconds-20s:v5'

### Specify the dataset artifact that we want to get the embeddings from
> If no artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [ ]:
input_ar_name = ifnone(
    config.input_ar, 
    f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}'
)
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'solar_4_seconds-20s:v5'

In [ ]:
df = input_ar.to_df()
df.head()

wandb:   1 of 1 files downloaded.  


,0,1,2,3,4
2002-01-01 00:00:00,4995.400909,3476.007487,3156.216405,1093.949562,302.794642
2002-01-01 00:00:20,3923.444920,2885.023318,2777.034856,789.073860,277.451848
2002-01-01 00:00:40,4129.310217,2733.710683,2884.988168,736.149227,301.431725
2002-01-01 00:01:00,5329.409519,2877.257957,3593.461371,780.097337,377.047302
2002-01-01 00:01:20,6150.810094,3060.637287,4012.004044,849.544364,406.177858


In [ ]:
df.shape

(46455, 5)

In [ ]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(102, 5, 900)

In [ ]:
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

--> Check CUDA
--> Ensure empty cache
--> Use CUDA |Get enc embs GPU 
CUDA está disponible
Dispositivo CUDA actual:  0
Nombre del dispositivo CUDA actual:  NVIDIA GeForce RTX 3090
--> Set dataset from X (enc_learn does not contain dls)
--> Get module
--> Get enc embs bs:  1
--> Concat
Fit in GPU
--> reduce
--> 2 numpy


In [ ]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [ ]:
run.finish()